# 04 — EDA avançado: narrativa e insights exportáveis

Objetivo: transformar EDA em portfólio: perguntas, evidências e insights registráveis.

Tempo: ~20–25 min

## Por que isso importa?

Uma EDA “bonita” não vira portfólio se ela não responde perguntas.

Aqui você vai:
- definir **perguntas de negócio**
- produzir **evidências (tabelas + gráficos)**
- registrar **insights em um formato exportável** (Markdown)

Você pode usar esse arquivo para “colar” no README do seu projeto.

Abra também o checklist: `templates/EDA_CHECKLIST.md`

In [ ]:
from __future__ import annotations

from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

def find_repo_root(start: Path | None = None) -> Path:
    """Sobe diretórios até achar README.md + pasta data."""
    cur = (start or Path.cwd()).resolve()
    for _ in range(10):
        if (cur / "README.md").exists() and (cur / "data").exists():
            return cur
        cur = cur.parent
    return Path.cwd().resolve()

root = find_repo_root()
DATA = root / "data"

In [ ]:
sales = pd.read_csv(DATA / "sample" / "sales.csv")
customers = pd.read_csv(DATA / "sample" / "customers.csv")

sales["date"] = pd.to_datetime(sales["date"])
customers["signup_date"] = pd.to_datetime(customers["signup_date"])

df = sales.merge(customers, on="customer_id", how="left")

df.head()

## 1) Perguntas (exemplo)

1- O que mais puxa receita: **categoria**, **região** ou **segmento**?  
2- Existe diferença de ticket médio entre **B2B vs B2C**?  
3- Quais produtos são “carregadores de receita” (Pareto 80/20)?

## 2) Evidências (tabelas e gráficos)

In [ ]:
# Ticket médio por segmento
seg = (
    df.groupby("segment")
      .agg(revenue_total=("revenue", "sum"), orders=("order_id", "nunique"))
      .assign(avg_ticket=lambda x: x["revenue_total"] / x["orders"])
      .sort_values("avg_ticket", ascending=False)
)
display(seg)

fig, ax = plt.subplots(figsize=(6, 4))
sns.barplot(data=seg.reset_index(), x="segment", y="avg_ticket", ax=ax)
ax.set_title("Ticket médio por segmento")
plt.show()

In [ ]:
# Pareto por produto (80/20)
prod = df.groupby("product")["revenue"].sum().sort_values(ascending=False)
pareto = (prod.cumsum() / prod.sum()).reset_index()
pareto.columns = ["product", "cum_share"]

k = int((pareto["cum_share"] <= 0.80).sum())
print("Produtos que acumulam ~80% da receita:", k)

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(range(1, len(pareto)+1), pareto["cum_share"])
ax.axhline(0.8, linestyle="--")
ax.set_title("Pareto: share acumulado de receita por produto")
ax.set_xlabel("rank do produto")
ax.set_ylabel("share acumulado")
plt.show()

## 3) Registrando insights em formato exportável (Markdown)

In [ ]:
insights = [
    {
        "pergunta": "Ticket médio difere entre segmentos?",
        "insight": f"Segmento líder em ticket médio: {seg.index[0]} (avg_ticket={seg['avg_ticket'].iloc[0]:.2f}).",
        "evidencia": "Tabela seg + gráfico de barras (ticket médio por segmento)"
    },
    {
        "pergunta": "Quantos produtos fazem ~80% da receita?",
        "insight": f"~{k} produtos acumulam aproximadamente 80% da receita.",
        "evidencia": "Curva de Pareto (share acumulado)"
    },
]

insights_df = pd.DataFrame(insights)
display(insights_df)

out_path = DATA / "output" / "insights_07_04.md"
out_path.parent.mkdir(parents=True, exist_ok=True)
out_path.write_text(insights_df.to_markdown(index=False), encoding="utf-8")
print("Salvo:", out_path)

## Exercícios (10–15 min)

1- Escreva 3 perguntas diferentes (ex.: crescimento, sazonalidade, mix).  
2- Para cada pergunta, gere 1 evidência (tabela ou gráfico).  
3- Registre os 3 insights no mesmo dataframe e exporte em Markdown.